In [2]:
#@title IMPORTING LIBRARIES
import zipfile
import pandas as pd
import numpy as np
import glob
import itertools
import networkx as nx

import string
import nltk
nltk.download('punkt_tab')
nltk.download('wordnet')
from nltk.tokenize import (sent_tokenize, word_tokenize)
from nltk import ngrams
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import regexp_tokenize
#Stop words
from nltk.corpus import stopwords
#Lemmatizer
#from nltk.stem import WordNetLemmatizer
!pip install unidecode
import unidecode

#Stemmer
#from nltk.stem import PorterStemmer

import re
import os
import sys
import time
import tracemalloc
from multiprocessing import Pool, cpu_count
from IPython.display import clear_output
# Clear the output
clear_output()

!pip install datasketch
from datasketch import MinHash, MinHashLSH


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 7.4 MB/s eta 0:00:00


In [3]:
#@title Importing credentials for Kaggle
os.environ['KAGGLE_USERNAME'] = "rodionhorbunov"
os.environ['KAGGLE_KEY'] = "e37ca7b26f7537fabcdf49a42416df7a"
!kaggle datasets download -d asaniczka/1-3m-linkedin-jobs-and-skills-2024

clear_output()

In [4]:
#@title Downloading dataset, unzip and extract all files, delete useless one

# Set the zip path
zip_path = '/content/1-3m-linkedin-jobs-and-skills-2024.zip'

# Specify the directory to extract the contents of the ZIP file to
extract_path = '/content/linkedin-jobs-and-skills'

# Open the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Extract all the contents of the ZIP file to the extract_path directory
    zip_ref.extractall(extract_path)


In [5]:
#@title Downloading dataset, unzip and extract all files
df = pd.read_csv('/content/linkedin-jobs-and-skills/job_summary.csv')

print(df.head(5))

# Check the duplicates
df_len = len(df)
df_len_wo_dupl = len(df.drop_duplicates())

print("Lenght - ",df_len, "Length w/o duplicates",df_len_wo_dupl,"\n")
#Almost 50 sec for this operations

#The same position and different linkdin posts

unique_positions = len(df["job_summary"].drop_duplicates())
print("There are",unique_positions," same position with different linkdin posts","\n")

#Identical positions (different post time)
duplicates_position = df_len - unique_positions
print("There are",duplicates_position,"identical positions","\n")

#Share of identical position
print("Share of identical position", (1297332-958192)/1297332,"\n")



                                            job_link  \
0  https://www.linkedin.com/jobs/view/restaurant-...   
1  https://www.linkedin.com/jobs/view/med-surg-re...   
2  https://www.linkedin.com/jobs/view/registered-...   
3  https://uk.linkedin.com/jobs/view/commercial-a...   
4  https://www.linkedin.com/jobs/view/store-manag...   

                                         job_summary  
0  Rock N Roll Sushi is hiring a Restaurant Manag...  
1  Schedule\n: PRN is required minimum 12 hours p...  
2  Description\nIntroduction\nAre you looking for...  
3  Commercial account executive\nSheffield\nFull ...  
4  Address:\nUSA-CT-Newington-44 Fenn Road\nStore...  
Lenght -  1297332 Length w/o duplicates 1297332 

There are 958192  same position with different linkdin posts 

There are 339140 identical positions 

Share of identical position 0.2614134238575785 



In [6]:
#@title  Fixing the indexes, mapping the dataset and providing the grouping (cleaning from duplicates)
df = df.reset_index()

result_df = df.groupby('job_summary').agg(
    min_index=('index', lambda x: x.index.min()),
    all_indexes=('index', lambda x: list(x.index))
)

In [7]:
#@title Grouped dataset check
result_df

min_index all_indexes
job_summary                                                              
! CURRENTLY SEEKING MULTIPLE RN SUPERVISORS, NO...    1010560   [1010560]
! URGENTLY HIRING !\nWe have an amazing opportu...     160323    [160323]
!! $20K Sign-On Bonus !!\nProvides professional...     566881    [566881]
!! Exciting New Opportunity !!\nJob Title\n:\nS...    1166832   [1166832]
!! NOW HIRING A CHEF !!\nMaple Ridge Grocer/Bis...    1072091   [1072091]
...                                                       ...         ...
🧩\nAbout us 🧩\nNoBa Capital is a hand-on invest...     417035    [417035]
🧬\nUnlock the Secrets of Drug Discovery: Senior...     472599    [472599]
🩺 Embark on a fulfilling career with this disti...    1071882   [1071882]
🪠 We invite full-time residential\nSenior HVAC ...     488809    [488809]
🪠 We invite full-time residential\nSenior Plumb...     584367    [584367]

[958192 rows x 2 columns]

In [8]:
#@title Processing new dataset
result_df = result_df.reset_index()
result_df["nbr_of_same_jobs"] = result_df["all_indexes"].apply(lambda x: len(x))

In [14]:
#@title Creating test sample of 10k rows

test_df = result_df[0:10000]

In [15]:
#@title Processing test dataset with straight forward computing
k = 5
threshold_similarity = 0.6

data =  test_df['job_summary']
# Create an MinHashLSH index optimized for Jaccard threshold 0.5,
# that accepts MinHash objects with 128 permutations functions
lsh = MinHashLSH(threshold=threshold_similarity, num_perm=32)

# Create MinHash objects
minhashes = {}
for c, i in enumerate(data):
  minhash = MinHash(num_perm=32)
  word_tokenized = word_tokenize(re.sub(r"[^\w\s]", " ", i.lower()), language="english", preserve_line=False)
  for j in range(len(word_tokenized) - k + 1):
    minhash.update(" ".join(word_tokenized[j:j + k]).encode('utf-8'))
  lsh.insert(test_df.loc[c,'min_index'], minhash)
  minhashes[test_df.loc[c,'min_index']] = minhash


#for i in range(len(minhashes.keys())):
#  result = lsh.query(minhashes[i])
#  print(f"Candidates with Jaccard similarity > {threshold_similarity} for input", i, ":", result)

In [18]:
#@title Processing test dataset with parallel computing

data =  test_df['job_summary']
k = 5
threshold_similarity = 0.6


# Define a function for processing each item
def process_item(args):
    c, i, result_row, k = args
    minhash = MinHash(num_perm=32)
    word_tokenized = word_tokenize(re.sub(r"[^\w\s]", " ", i.lower()), language="english", preserve_line=False)
    for j in range(len(word_tokenized) - k + 1):
        minhash.update(" ".join(word_tokenized[j:j + k]).encode('utf-8'))
    return result_row['min_index'], minhash

# Initialize LSH and MinHashes
minhashes = {}
lsh = MinHashLSH(threshold=threshold_similarity, num_perm=32)

# Set your desired value of k
data_with_indices = [(c, i, test_df.loc[c], k) for c, i in enumerate(data)]

# Use multiprocessing to process the data
with Pool(processes=cpu_count()) as pool:
    results = pool.map(process_item, data_with_indices)

# Update LSH and MinHashes after parallel processing
for min_index, minhash in results:
    lsh.insert(min_index, minhash)
    minhashes[min_index] = minhash


In [21]:
#@title Check memory usage of all variables
for var_name, var_obj in sorted(vars().items()):
  print(f"{var_name}: {sys.getsizeof(var_obj)/1024**2:.2f} MB")

In: 0.00 MB
MinHash: 0.00 MB
MinHashLSH: 0.00 MB
Out: 0.00 MB
Pool: 0.00 MB
RegexpTokenizer: 0.00 MB
_: 8524.35 MB
_11: 8524.35 MB
_7: 8517.04 MB
__: 8517.04 MB
___: 0.00 MB
__builtin__: 0.00 MB
__builtins__: 0.00 MB
__doc__: 0.00 MB
__loader__: 0.00 MB
__name__: 0.00 MB
__package__: 0.00 MB
__spec__: 0.00 MB
_dh: 0.00 MB
_exit_code: 0.00 MB
_i: 0.00 MB
_i1: 0.00 MB
_i10: 0.00 MB
_i11: 0.00 MB
_i12: 0.00 MB
_i13: 0.00 MB
_i14: 0.00 MB
_i15: 0.00 MB
_i16: 0.00 MB
_i17: 0.00 MB
_i18: 0.00 MB
_i19: 0.00 MB
_i2: 0.00 MB
_i20: 0.00 MB
_i21: 0.00 MB
_i3: 0.00 MB
_i4: 0.00 MB
_i5: 0.00 MB
_i6: 0.00 MB
_i7: 0.00 MB
_i8: 0.00 MB
_i9: 0.00 MB
_ih: 0.00 MB
_ii: 0.00 MB
_iii: 0.00 MB
_oh: 0.00 MB
c: 0.00 MB
clear_output: 0.00 MB
cpu_count: 0.00 MB
data: 80.40 MB
data_with_indices: 0.08 MB
df_len: 0.00 MB
df_len_wo_dupl: 0.00 MB
duplicates_position: 0.00 MB
exit: 0.00 MB
extract_path: 0.00 MB
get_ipython: 0.00 MB
glob: 0.00 MB
i: 0.01 MB
itertools: 0.00 MB
j: 0.00 MB
k: 0.00 MB
lsh: 0.00 MB
min_ind

In [20]:
#@title Delete downloaded dataset
del df

In [22]:
#@title Processing full dataset
data =  result_df['job_summary']
k = 5
threshold_similarity = 0.6


# Define a function for processing each item
def process_item(args):
    c, i, result_row, k = args
    minhash = MinHash(num_perm=32)
    word_tokenized = word_tokenize(re.sub(r"[^\w\s]", " ", i.lower()), language="english", preserve_line=False)
    for j in range(len(word_tokenized) - k + 1):
        minhash.update(" ".join(word_tokenized[j:j + k]).encode('utf-8'))
    return result_row['min_index'], minhash

# Initialize LSH and MinHashes
minhashes = {}
lsh = MinHashLSH(threshold=threshold_similarity, num_perm=32)

# Set your desired value of k
data_with_indices = [(c, i, result_df.loc[c], k) for c, i in enumerate(data)]

# Use multiprocessing to process the data
with Pool(processes=cpu_count()) as pool:
    results = pool.map(process_item, data_with_indices)

# Update LSH and MinHashes after parallel processing
for min_index, minhash in results:
    lsh.insert(min_index, minhash)
    minhashes[min_index] = minhash


#40 minutes of computation time

In [44]:
#@title Check the results

index_to_check = 891049

same_job_descriptions = set(lsh.query(minhashes[index_to_check]) +result_df.loc[index_to_check,'all_indexes'])

print(same_job_descriptions)

for index in same_job_descriptions:
  print(result_df[result_df['min_index']==index]['job_summary'])

{891049, 891009, 1037834}
997    "You can make a difference in the life of an o...
Name: job_summary, dtype: object
998    "You can make a difference in the life of an o...
Name: job_summary, dtype: object
891049    We are looking for a Associate Assistant Head ...
Name: job_summary, dtype: object


In [41]:
#@title Groupung by the non-unique jobs
result_df.sort_values('nbr_of_same_jobs',ascending = False).head(20)

,job_summary,min_index,all_indexes,nbr_of_same_jobs
275426,Dollar General Corporation has been delivering...,23514,"[23514, 23670, 23691, 23794, 23891, 23892, 242...",4571
769230,Store Family Dollar\nFamily Dollar is seeking ...,47060,"[47060, 47121, 47165, 47177, 47252, 47289, 473...",4132
81311,Are you looking for a job that offers more res...,4417,"[4417, 7853, 13298, 13633, 19850, 20426, 20569...",2489
81303,Are you looking for a job that offers more res...,48033,"[48033, 48983, 49017, 49075, 49083, 49444, 496...",2108
238223,Description\nPLEASE NOTE:\nYour application wi...,7540,"[7540, 7909, 8029, 8213, 8831, 27280, 27703, 4...",2028
769221,Store Dollar Tree\nWork where you love to shop...,18390,"[18390, 19950, 22163, 22226, 22394, 22487, 224...",1984
390679,"Job Description\nDepartment Supervisors lead, ...",31680,"[31680, 52613, 52942, 55068, 55121, 55481, 555...",1549
831796,This job posting is for a position in a restau...,4418,"[4418, 4686, 7279, 7364, 7517, 7800, 9016, 175...",1368
275539,Dollar General Corporation has been delivering...,12,"[12, 22, 47, 54, 61, 86, 122, 142, 156, 163, 1...",1245
607790,Overview\nSALLY BEAUTY ADVISOR:\nJob Descripti...,212527,"[212527, 212529, 212531, 212551, 212553, 21256...",1138


In [47]:
#@title Check the results 2

index_to_check = 11939

same_job_descriptions = set(lsh.query(minhashes[index_to_check]) +result_df.loc[index_to_check,'all_indexes'])

for index in same_job_descriptions:
  print(result_df[result_df['min_index']==index]['job_summary'])

275499    Dollar General Corporation has been delivering...
Name: job_summary, dtype: object
275488    Dollar General Corporation has been delivering...
Name: job_summary, dtype: object
275517    Dollar General Corporation has been delivering...
Name: job_summary, dtype: object
275497    Dollar General Corporation has been delivering...
Name: job_summary, dtype: object
275405    Dollar General Corporation has been delivering...
Name: job_summary, dtype: object
275514    Dollar General Corporation has been delivering...
Name: job_summary, dtype: object
275503    Dollar General Corporation has been delivering...
Name: job_summary, dtype: object
275494    Dollar General Corporation has been delivering...
Name: job_summary, dtype: object
275498    Dollar General Corporation has been delivering...
Name: job_summary, dtype: object
275496    Dollar General Corporation has been delivering...
Name: job_summary, dtype: object
275518    Dollar General Corporation has been delivering...
Name: job_

In [9]:
#@title Installing packages

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark

!pip install pyspark
!pip install findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Start Spark session
spark = SparkSession.builder.appName("MinHashLSH from Text").getOrCreate()

from pyspark.context import SparkContext
sc = SparkContext.getOrCreate()




# IMPORTING LIBRARIES
import zipfile
import pandas as pd
import numpy as np
import glob
import itertools
import networkx as nx
import matplotlib.pyplot as plt
from functools import reduce
from pyspark.sql.functions import lit, collect_set, col, broadcast, expr, collect_list, countDistinct, count
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import ArrayType, StructType, StructField, StringType
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col
from pyspark.ml.feature import MinHashLSH
from pyspark.sql import SparkSession
from pyspark.ml.feature import HashingTF, IDF, CountVectorizer
from pyspark.ml.linalg import Vectors
from pyspark.sql import Row
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import udf, col
from pyspark.sql.types import ArrayType, StringType

#Import packages to work with text data

import string
import nltk
nltk.download('punkt_tab')
nltk.download('wordnet')
from nltk.tokenize import (sent_tokenize, word_tokenize)
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import regexp_tokenize
#Stop words
from nltk.corpus import stopwords
#Lemmatizer
from nltk.stem import WordNetLemmatizer
!pip install unidecode
import unidecode

#Stemmer
#from nltk.stem import PorterStemmer

import re

import os
import sys
import time
import tracemalloc

from IPython.display import clear_output
# Clear the output
clear_output()

In [10]:
result_df.head(5)

,job_summary,min_index,all_indexes,nbr_of_same_jobs
0,"! CURRENTLY SEEKING MULTIPLE RN SUPERVISORS, N...",1010560,[1010560],1
1,! URGENTLY HIRING !\nWe have an amazing opport...,160323,[160323],1
2,!! $20K Sign-On Bonus !!\nProvides professiona...,566881,[566881],1
3,!! Exciting New Opportunity !!\nJob Title\n:\n...,1166832,[1166832],1
4,!! NOW HIRING A CHEF !!\nMaple Ridge Grocer/Bi...,1072091,[1072091],1


In [71]:
#Saving the sample from dataset to upload it to pyshark
result_df[0:10000].to_csv("result.csv", index=False)

In [49]:
#Uploading dataframe
df = spark.read.csv("result.csv", header=True,multiLine=True,
    escape='"',        # Escape character for quotes
    quote='"',         # Quote character to handle multi-line strings
    inferSchema=True   # Infer data types
                    )

In [50]:
#Checking dataframe
df.show(10)

+--------------------+---------+-----------------+----------------+
|         job_summary|min_index|      all_indexes|nbr_of_same_jobs|
+--------------------+---------+-----------------+----------------+
|! CURRENTLY SEEKI...|  1010560|        [1010560]|               1|
|! URGENTLY HIRING...|   160323|         [160323]|               1|
|!! $20K Sign-On B...|   566881|         [566881]|               1|
|!! Exciting New O...|  1166832|        [1166832]|               1|
|!! NOW HIRING A C...|  1072091|        [1072091]|               1|
|!! NOW HIRING SUP...|   603275|         [603275]|               1|
|!! Yokozuna on Ya...|   109598|         [109598]|               1|
|!!!!!!!!!!!!!!!!!...|   207591|         [207591]|               1|
|!!!WEEKLY PAY STA...|   594005|[594005, 1004174]|               2|
|!!New Position Al...|   143461|         [143461]|               1|
+--------------------+---------+-----------------+----------------+
only showing top 10 rows



In [51]:
#Checking dataframe
df.count()

10000

In [52]:
#@title Tokenize text by words
tokenizer = Tokenizer(inputCol="job_summary", outputCol="words")
tokenized = tokenizer.transform(df)

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=250)
featurizedData = hashingTF.transform(tokenized)

countVectorizer = CountVectorizer(inputCol="words", outputCol="rawFeatures")
model = countVectorizer.fit(tokenized)
featurizedData = model.transform(tokenized)

In [53]:
#@title Featurized data and MinHashLSH (Tokenize text by words)
featurizedData.select("min_index", "rawFeatures").show(truncate=False)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

df_new = rescaledData.select("min_index", "features").show(truncate=False)


df_new = rescaledData.select("min_index", "features")
df_new.show(truncate=False)

mh = MinHashLSH()
mh.setInputCol("features")
mh.setOutputCol("hashes")
mh.setSeed(12345)
model = mh.fit(df_new)
model.setInputCol("features")
model.setInputCol("features")


+---------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

MinHashLSHModel: uid=MinHashLSH_0da8a2f0d25f, numHashTables=1

In [54]:
#@title Similarity estimation (Tokenize text by words)

model.approxSimilarityJoin(df_new, df_new, 0.5, distCol="JaccardDistance").select(
    col("datasetA.min_index").alias("min_indexA"),
    col("datasetB.min_index").alias("min_indexB"),
    col("JaccardDistance")).createOrReplaceTempView("similarity_view_words")

In [69]:
#@title Similarity examples (Tokenize text by words)

jacard_dist_approx = 0.5
index = 1150446
result = spark.sql(f"SELECT min_indexA,min_indexB FROM similarity_view_words where JaccardDistance < {jacard_dist_approx} and min_indexA = {index} LIMIT 10")
result.show()


+----------+----------+
|min_indexA|min_indexB|
+----------+----------+
|   1150446|   1111016|
|   1150446|   1150446|
|   1150446|    522857|
|   1150446|    674562|
|   1150446|   1110634|
+----------+----------+



In [72]:
#@title Tokenize text by sequence of words

result_df[0:100000].to_csv("result.csv", index=False)

df = spark.read.csv("result.csv", header=True,multiLine=True,
    escape='"',        # Escape character for quotes
    quote='"',         # Quote character to handle multi-line strings
    inferSchema=True   # Infer data types
                    )

In [101]:
#@title Tokenize text by sequence of words , creating ngrams, vectorizing

from pyspark.sql.functions import udf

# Tokenize text by sequence of words
tokenizer = Tokenizer(inputCol="job_summary", outputCol="words")
tokenized = tokenizer.transform(df)

# Define a UDF to generate N-grams
k = 3

#Create the function with ngram
def generate_ngrams(words):
    return [" ".join(words[i:i+k]) for i in range(len(words) - k +1 )]

# Register UDF
generate_ngrams_udf = udf(generate_ngrams, ArrayType(StringType()))

# Apply the UDF to create a new column for trigrams
ngram_df = tokenized.withColumn("ngram", generate_ngrams_udf(col("words")))
ngram_df.select("min_index", "ngram").show(truncate=False)

#Hashing ngrams
hashingTF = HashingTF(inputCol="ngram", outputCol="rawFeatures", numFeatures=250, binary = True) #, binary = True
featurizedData = hashingTF.transform(ngram_df)
featurizedData = hashingTF.transform(ngram_df)

#Creating the vectors
countVectorizer = CountVectorizer(inputCol="ngram", outputCol="rawFeatures") #,vocabSize=10000, minDF=2
model = countVectorizer.fit(ngram_df)
featurizedData = model.transform(ngram_df)

+---------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [102]:
#Creating new df before modeling
df_new = featurizedData.select("min_index", "rawFeatures")
df_new.show(truncate=False)


+---------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [103]:
#@title Modeling sequence of words

#Modeling
mh = MinHashLSH()
mh.setInputCol("rawFeatures")
mh.setOutputCol("hashes")

#Set seeds
mh.setSeed(12345)

#Fit modeling
model = mh.fit(df_new)
model.setInputCol("rawFeatures")
model.setInputCol("features")


MinHashLSHModel: uid=MinHashLSH_5f992562ec87, numHashTables=1

In [104]:
#@title Modeling sequence of words

model.approxSimilarityJoin(df_new, df_new, 0.5, distCol="JaccardDistance").select(
    col("datasetA.min_index").alias("min_indexA"),
    col("datasetB.min_index").alias("min_indexB"),
    col("JaccardDistance")).createOrReplaceTempView("similarity_view_seq")

In [105]:
#@title Examples of pairs

jacard_dist_approx = 0.5
index = 1150446
result = spark.sql(f"SELECT min_indexA,min_indexB FROM similarity_view_seq where min_indexA = {index}  Limit 5")
result.show()

+----------+----------+
|min_indexA|min_indexB|
+----------+----------+
|   1150446|   1110634|
|   1150446|   1150446|
|   1150446|    674562|
|   1150446|    522857|
|   1150446|   1111016|
+----------+----------+

